### 끝말잇기 게임

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                            google_api_key=gemini_api_key)

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 
            아래는 게임 규칙입니다.
            당신과 user의 입력에서 아래 규칙이 꼭 지켜져야 하며,
            지키지 않은 사람에게 패배를 알린 뒤,
            끝말잇기 게임을 종료합니다.
            1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
            2. 두음법측을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
            3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
            4. 아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력하세요.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

In [15]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [16]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요."
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    chat_history.clear()

    chat_history.add_message(summary_message)

    print("summary_message:", summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [ ]:
# chain_with_summarization.invoke(
#         {"input": "랭체인"},
#         {"configurable": {"session_id": "unused"}},
#     ).content

'인도'

In [ ]:
while True:
    user_input = input("👨YOUR TURN: ")
    
    if user_input == "종료": break

    response = chain_with_summarization.invoke(
        {"input": user_input},
        {"configurable": {"session_id": "unused"}},
    )

    print("🤖AI TURN:", response.content)

